In [308]:
from qiskit.circuit import QuantumCircuit, QuantumRegister
import ffsim
import math
import numpy as np
from passive_extended_matchgate_simulator.raw_estimation import raw_estimate
from passive_extended_matchgate_simulator.utils import ucj_to_compatible, ucj_to_compatible_fully_reduced, calculate_trajectory_count, extract_circuit_data

norb = 3
nelec = (2, 2)
alpha_alpha_indices = [(p, p + 1) for p in range(norb - 1)]
alpha_beta_indices = [(p, p) for p in range(norb)]
qubits = QuantumRegister(2 * norb)
circuit = QuantumCircuit(qubits)
ucj_op = ffsim.random.random_ucj_op_spin_balanced(norb, 
                                                  interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
                                                  with_final_orbital_rotation=False,
                                                  diag_coulomb_mean=math.pi, 
                                                  diag_coulomb_scale=.1, 
                                                  diag_coulomb_normal=True)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec), qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)

statevec = ffsim.qiskit.final_state_vector(circuit)
ffsim_probs = np.abs(statevec.vec)**2
bitstrings = ffsim.addresses_to_strings(range(len(ffsim_probs)), norb, nelec)
ffsim_p_and_b = list(zip(ffsim_probs, bitstrings))

epsilon = .1
delta = .01
p = 1

compatible = ucj_to_compatible_fully_reduced(circuit)

In [309]:
ffsim_p_and_b

[(np.float64(0.31922917493662883), 27),
 (np.float64(0.09835597335880929), 43),
 (np.float64(0.09588104188035901), 51),
 (np.float64(0.09835597335880938), 29),
 (np.float64(0.25348637525113665), 45),
 (np.float64(0.013640017878079886), 53),
 (np.float64(0.09588104188035901), 30),
 (np.float64(0.013640017878079886), 46),
 (np.float64(0.011530383577740777), 54)]

In [310]:
raw_estimate_values = []
for b in bitstrings:
    raw_estimate_values.append(raw_estimate(circuit=compatible,outcome_state=b, epsilon=epsilon, delta=delta, p=p))
raw_estimate_values

[0.3286350435323587,
 0.09884398093610709,
 0.09479306076426823,
 0.10155429083446238,
 0.25000654431926683,
 0.013906097036009568,
 0.09454453051353827,
 0.014158949369657593,
 0.011890827985815663]

In [311]:
diff = np.abs(ffsim_probs - raw_estimate_values)
print("max difference", max(diff))
data = extract_circuit_data(compatible)
extent = data[0]
t = calculate_trajectory_count(epsilon, delta, extent, p)
print("trajectory count", t)
diff


max difference 0.009405868595729838
trajectory count 925942


array([0.00940587, 0.00048801, 0.00108798, 0.00319832, 0.00347983,
       0.00026608, 0.00133651, 0.00051893, 0.00036044])